## Explication de notre projet d'intégration des données
### 🥘 - Différents type de régimes : 
<b>• Foodmap :</b> Favorise certains aliments contenant des glucides capable de fermenter dans l'intestin. <br>

<b>• Méditerranéen :</b> Favorise l'équilibre alimentaire, pour privilégier la variété des aliments. <br>

<b>• Dash (Dietary Approaches to Stopping Hypertension) :</b> Permet de combattre l'hypertension artérielle en modifiant les habitudes alimentaires.<br>

<b>• À indice glycémique bas :</b> Régule le taux de sucre dans le sang en fonction des aliments et deux leur indice glycémique (IG).<br>

<b>• Végétarien/Végétalisme :</b> Exclut toutes chaires animales / Exclut la consommation de tous produits d'origine animale.<br>

<b>• Paléolithique :</b> Favorise la même alimentation que nos ancêtres, avant l'ère industrielle. <br>

<b>• Cétogène :</b> Suppresion quasi complète des sucres et amidons(glucides).<br>

### 👔 - Étapes d'avancements 
<b>• Étape 1 :</b> Importation des libraries et des données<br>

<b>• Étape 2 :</b> Nettoyage des données importées<br>

<b>• Étape 3 :</b> Génération des menus, en fonction des régimes alimentaires des utilisateurs<br>

<b>• Étape 4 :</b> Extraction des données vers la base de données<br>

### <b>Étape 1</b> : Importation des libraries et des données
<hr/>

### 📚 - Importation de les libraries nécessaires pour le code 

Les imports de PySpark, notamment SparkSession pour la gestion de la session et les fonctions col et expr pour la manipulation des DataFrames, ont été réalisés pour permettre des opérations efficaces sur les données.

In [16]:
# Import nécessaires des packages 
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, expr
import sqlite3

### 💽 - Importation de l'ensemble des données 

L'ensemble des données a été importé et stocké dans des DataFrames distincts, facilitant ainsi la manipulation et l'analyse ultérieure des informations relatives aux produits alimentaires et aux pays.

In [17]:
# Création d'une session Spark, entrée principale pour intéragir avec pyspark
spark = SparkSession.builder.appName("data_integration_openfoodfacts").getOrCreate()

# Importation des données openfoodfacts dans le dataframe, délimitation sur la tabulation et récupération de l'en-tête 
df_openfoodfacts = spark.read.option("delimiter", "\t").csv("../data/en.openfoodfacts.org.products.csv", header=True, inferSchema=True)

# Sélectionne seulement les colonnes qui nous intéressent pour la suite 
df_openfoodfacts = df_openfoodfacts.select(["product_name", "brands", "categories", "countries", "energy-kcal_100g", "proteins_100g", "fat_100g", "carbohydrates_100g"])

# Importation des données de régimes 
df_diets = spark.read.csv("../data/diets.csv", header=True, inferSchema=True)

# Importation des données des utilisateurs 
df_users = spark.read.csv("../data/users.csv", header=True, inferSchema=True)

# Importation des données de tous les pays du mondes, pour pouvoir avoir un référentiel propre 
df_pays = spark.read.csv("../data/country.csv", header=True, inferSchema=True)

# Importation des données de jour, afin de décrire une semaine 
df_jours_semaine = spark.read.csv("../data/days.csv", header=True, inferSchema=True)

# Conserve seulement les colonnes qui nous intèresse pour la suite des traitements 
df_pays = df_pays.select(["id", "country"])

### 📊 - Affichage du schéma du DataFrame avec colonnes et types de données

Le schéma du DataFrame, affichant les colonnes ainsi que leurs types de données, a été affiché pour offrir une vue d'ensemble des informations structurées.

In [18]:
# Affiche le schéma du dataframe, indiquant les colonnes et leurs types de données 
df_openfoodfacts.printSchema()

root
 |-- product_name: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- countries: string (nullable = true)
 |-- energy-kcal_100g: double (nullable = true)
 |-- proteins_100g: double (nullable = true)
 |-- fat_100g: double (nullable = true)
 |-- carbohydrates_100g: double (nullable = true)



### <b>Étape 2</b> : Nettoyage des données importées 
<hr/>

### 🗑️ - Suppression des valeurs nulles

Dans cette section, nous avons supprimé les lignes avec des valeurs nulles puis filtré les produits en veillant à ce que des colonnes clés ne contiennent pas de valeurs nulles, préservant ainsi l'intégrité des données. 

In [19]:
# Supprimer les lignes ayant des valeurs nulles 
df_openfoodfacts.dropna()

# Filtrer les produits ayant des valeurs nulles 
df_openfoodfacts = df_openfoodfacts.filter(
    col("product_name").isNotNull() &
    col("categories").isNotNull() &
    col("countries").isNotNull() &
    col("energy-kcal_100g").isNotNull() &
    col("proteins_100g").isNotNull() &
    col("fat_100g").isNotNull() &
    col("carbohydrates_100g").isNotNull()
)

### 🗑️ - Suppresion des valeurs aberrantes

Dans ce code, nous avons établi des plages de valeurs minimales et maximales pour certaines données d'intérêt. 

<b>Énergie (kcal/100g)</b> : Les valeurs caloriques par portion devraient se situer entre 0 kcal/100g (minimum) et 800 kcal/100g (maximum) pour être considérées comme non aberrantes.

<b>Protéines (g/100g)</b> : La teneur en protéines par portion serait considérée comme non aberrante si elle se situe dans la plage de 1 g/100g (minimum) à 100 g/100g (maximum).

<b>Graisses (g/100g)</b> : Les quantités de graisses par portion devraient être comprises entre un minimum de 0 g/100g et un maximum de 100 g/100g pour être considérées comme non aberrantes.

<b>Glucides (g/100g)</b> : La teneur en glucides par portion serait jugée non aberrante si elle se situe entre 1 g/100g (minimum) et 80 g/100g (maximum).

In [20]:
# Définir des valeurs spécifiques pour chaque colonne
valeurs_specifiques = {
    'energy-kcal_100g': (0, 800),
    'proteins_100g': (1, 100),
    'fat_100g': (0, 100),
    'carbohydrates_100g': (1, 80)
}

# Filtrer les produits qui contiennent des valeurs aberrantes pour les informations nutritionnelles
df_openfoodfacts = df_openfoodfacts.filter(
    (col("energy-kcal_100g") > valeurs_specifiques['energy-kcal_100g'][0]) &
    (col("energy-kcal_100g") <= valeurs_specifiques['energy-kcal_100g'][1]) &
    (col("proteins_100g") > valeurs_specifiques['proteins_100g'][0]) &
    (col("proteins_100g") <= valeurs_specifiques['proteins_100g'][1]) &
    (col("fat_100g") > valeurs_specifiques['fat_100g'][0]) &
    (col("fat_100g") <= valeurs_specifiques['fat_100g'][1]) &
    (col("carbohydrates_100g") > valeurs_specifiques['carbohydrates_100g'][0]) &
    (col("carbohydrates_100g") <= valeurs_specifiques['carbohydrates_100g'][1])
)

 ### 🧶 - Consolidation des données 

Dans ce bout de code, on fusionne les informations des produits alimentaires avec les données spécifiques des pays via une jointure <b>"left outer join"</b>. 

Ensuite on sélectionne les colonnes pertinentes, filtre les produits sans correspondance dans les pays, et élimine les doublons.</p>

In [21]:
# Effectuer une jointure left outer join
joined_df = df_openfoodfacts.join(df_pays, df_openfoodfacts["countries"] == df_pays["country"], "left_outer")

# Sélectionner les colonnes qui nous intéressent et filtrer les lignes où il n'y a pas de correspondance dans df_pays
df_openfoodfacts = joined_df.select(["product_name", "brands", "categories", "countries", "energy-kcal_100g", "proteins_100g", "fat_100g", "carbohydrates_100g"]).filter(col("countries").isNotNull())

# Supprimer les doublons
df_openfoodfacts = df_openfoodfacts.dropDuplicates(["product_name", "countries"])

# Afficher le dataframe 
df_openfoodfacts.show()

+--------------------+--------------------+--------------------+--------------------+----------------+-------------+--------+------------------+
|        product_name|              brands|          categories|           countries|energy-kcal_100g|proteins_100g|fat_100g|carbohydrates_100g|
+--------------------+--------------------+--------------------+--------------------+----------------+-------------+--------+------------------+
| Filet de merlu c...|    Très bien merci!|Plats préparés, P...|              France|            68.0|          7.9|     2.1|               4.3|
|&quot;CRISPY&quot...|              Melvit|Növényi alapú éle...|        Magyarország|           457.0|         20.0|    29.0|              20.0|
|&quot;La Traditio...|  Crêperie Le Masson|Crêpes et galette...|              France|           320.0|          7.0|     4.2|              64.0|
|&quot;Les Mijotés...|                NULL|Viandes et dérivé...|              France|           150.0|         21.0|     7.1|     

### <b>Étape 3</b> : Génération des menus, en fonction des régimes alimentaires des utilisateurs
<hr/>

 ### 🍲 - Création des menus

Dans ce bout de code, on génère un petit-déjeuner, un déjeuner et un dîner pour chaque jour de la semaine par utilisateurs. En fonction de son régime alimentaire.

In [22]:
# Liste pour stocker les résultats
weekly_menu = []

# Boucle pour chaque utilisateur
for utilisateur in df_users.collect():
    # Récupérer les valeurs spécifiques au régime dans le DataFrame des régimes
    regime_utilisateur = df_diets.filter(col("Diet_type") == utilisateur["diet"]).first()
    
    # Boucle pour chaque jour de la semaine
    for jour in df_jours_semaine.collect():
        # Filtrer les produits disponibles qui respectent les valeurs du régime alimentaire
        filtered_products = df_openfoodfacts.filter(
            (col("carbohydrates_100g") <= regime_utilisateur["carbohydrates_100g"]) &
            (col("proteins_100g") >= regime_utilisateur["proteins_100g"]) &
            (col("fat_100g") <= regime_utilisateur["fat_100g"]) &
            (col("energy-kcal_100g") <= regime_utilisateur["energy-kcal_100g"])
        )

        # Récupération des produits pour chaques repas de la journée 
        breakfast = filtered_products.sample(withReplacement=False, fraction=0.1).first()
        lunch = filtered_products.sample(withReplacement=False, fraction=0.1).first()
        dinner = filtered_products.sample(withReplacement=False, fraction=0.1).first()

            # Création de l'objet récupéré par la suite contenant l'ensemble des informations 
        daily_menu = {
            "id_user": utilisateur["id"],
            "day": jour["day"],
            "breakfast": breakfast.product_name,
            "lunch": lunch.product_name,
            "dinner": dinner.product_name
        }

        # Ajoute le menu dans la liste des menus globales 
        weekly_menu.append(daily_menu)

### <b>Étape 4</b> : Insertion des menus dans la base de données, étape d'extraction
<hr/>

 ### 📉 - Extraction des données 

Ici, on extrait les données dans la base de données, on se connecte on insère puis déconnexion.

In [23]:
# Connexion à la base de données sqlite3
conn = sqlite3.connect('../weekly_menu.db')

# Création d'un curseur
cursor = conn.cursor()

# Définir la commande SQL d'insertion
insert_command = '''
INSERT INTO daily_menu (id_user, day, breakfast, lunch, dinner)
VALUES (?, ?, ?, ?, ?);
'''

# Utiliser une transaction pour regrouper les insertions
try:
    for menu in weekly_menu:
        # Exécuter la commande SQL avec les données à insérer 
        cursor.execute(insert_command, (menu['id_user'], menu['day'], menu['breakfast'], menu['lunch'], menu['dinner']))

    # Commiter les changements à la fin de la transaction
    conn.commit()

except Exception as e:
    # En cas d'erreur, annuler la transaction
    conn.rollback()
    print(f"Une erreur s'est produite : {e}")

finally:
    # Fermeture du curseur et de la connexion à la base de données 
    cursor.close()
    conn.close()

 ### 🔚 - Fin des traitements

Ensuite, on clos la session spark.

In [6]:
# Arrête la session Spark 
spark.stop()